### Perform transfer learning to identify real and fake faces

In [1]:
#Transformer model: https://huggingface.co/facebook/deit-small-patch16-224

from transformers import AutoImageProcessor, AutoModel

processor = AutoImageProcessor.from_pretrained("facebook/deit-small-patch16-224", force_download=True)
model = AutoModel.from_pretrained("facebook/deit-small-patch16-224", force_download=True, add_pooling_layer=False)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/home/danny/git_projects/deep_learning_projects/deep_learning_projects/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/88.3M [00:00<?, ?B/s]

In [2]:
#Images from https://www.kaggle.com/datasets/hamzaboulahia/hardfakevsrealfaces

import os
base_path = '../data/faces'

file_list = []

for root, _, files in os.walk(base_path, topdown=False):
   for name in files:
      file_list.append(os.path.join(root, name))

file_list.sort()

#Create Test and Train Set

X = []
labels = []

for file in file_list:
    splitted = file.split('/')

    label = splitted[3]

    X.append(file)
    labels.append(label)



In [3]:
#Create mapping of labels to indicies

label_to_id = dict()

labs = list(set(labels))
labs.sort()

for ind, i in enumerate(labs):
    label_to_id[i] = ind

In [4]:
label_to_id

{'fake': 0, 'real': 1}

In [5]:
#Create label set for training

y = []

for i in labels:
    y.append(label_to_id[i])


In [6]:
from sklearn.model_selection import train_test_split

#Create Train, Val and Test Sets


X_train, X_o, y_train, y_o = train_test_split(X, y, test_size=0.2, random_state=111)
X_test, X_val, y_test, y_val = train_test_split(X_o, y_o, test_size=0.5, random_state=112)


In [7]:
len(X_train), len(X_val), len(X_test)

(1031, 129, 129)

In [8]:
from PIL import Image

path = X_train[0]
image = Image.open(path)
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [9]:
out_shape = outputs.last_hidden_state.shape
out_shape

torch.Size([1, 197, 384])

In [10]:
#Split to avoid memory errors
import torch

split_size = 64

def split(images, dim):
    temp = []

    i = 0

    while i < len(images):

        with torch.no_grad():

            inputs = processor(images=images[i:i+split_size], return_tensors="pt")
            outputs = model(**inputs)
            lhs = outputs.last_hidden_state[:, 0, :] #[CLS] token
            inputs_proc = lhs.view(inputs['pixel_values'].shape[0], dim)

        temp.append(inputs_proc)
        i += split_size

    return temp

### Train softmax on labels of output

In [11]:
from torch.nn import functional as F
from torch import nn
import numpy as np

dim2 = out_shape[2] #Output dimension (for passing to classifier)

X_train2 = np.array(X_train)
y_train2 = torch.tensor(np.array(y_train))

X_val2 = np.array(X_val)
y_val2 = torch.tensor(np.array(y_val))

#Store indices for batching later

train_inds = np.arange(len(X_train2))

#Pre-pass through all images for speed

#Train

images = []

for path in X_train2:
    image = Image.open(path)
    images.append(image)

#Validation

train_inputs = torch.cat(split(images, dim2))

images = []

for path in X_val2:
    image = Image.open(path)
    images.append(image)

#Pass through transformer

val_inputs = torch.cat(split(images, dim2))

#Define model

class Model(nn.Module):

    def __init__(self, dim2, out_dim):
        super().__init__()

        #Softmax Classification Layer
        self.linear = nn.Linear(in_features=dim2, out_features=out_dim, bias=False) #Linear layer 

    def forward(self, X, targets):
        
        logits = self.linear(X)

        loss = F.cross_entropy(logits, targets)

        return logits, loss

In [12]:
batch_size = 64
iterations = 2500
print_cadence = 50
learning_rate = 0.001

#Set random seed
np.random.seed(0)
torch.manual_seed(1)

#Instantiate model

classifier = Model(dim2, len(label_to_id))
classifier.to('cuda')

#Define optimizer

optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

#Perform training

for iter in range(iterations):

    #Output as desired

    if iter % print_cadence == 0:

        with torch.no_grad():

            #Set to eval mode
            classifier.eval()

            #Compute train/val losses
            _, train_loss = classifier(train_inputs.to('cuda'), y_train2.to('cuda'))
            _, val_loss = classifier(val_inputs.to('cuda'), y_val2.to('cuda'))

            print(f'iter: {iter} train_loss: {train_loss} val_loss: {val_loss}')

            #Set back to train mode
            classifier.train()

    #Zero gradients

    classifier.zero_grad()

    #Get random batch

    batch_ind = np.random.choice(train_inds, size=batch_size, replace=False)

    #Pass model forward

    _, loss = classifier(train_inputs[batch_ind].to('cuda'), y_train2[batch_ind].to('cuda'))

    #Call Backward to get gradients

    loss.backward()

    #Perform optimization

    optimizer.step()

iter: 0 train_loss: 0.6830078363418579 val_loss: 0.7005904316902161
iter: 50 train_loss: 0.22814179956912994 val_loss: 0.21171599626541138
iter: 100 train_loss: 0.15702001750469208 val_loss: 0.1563359797000885
iter: 150 train_loss: 0.1205873042345047 val_loss: 0.12530486285686493
iter: 200 train_loss: 0.09923643618822098 val_loss: 0.10867346823215485
iter: 250 train_loss: 0.084819495677948 val_loss: 0.1045040488243103
iter: 300 train_loss: 0.07252571731805801 val_loss: 0.09284936636686325
iter: 350 train_loss: 0.06347836554050446 val_loss: 0.0849367305636406
iter: 400 train_loss: 0.05637827515602112 val_loss: 0.0818326473236084
iter: 450 train_loss: 0.05058078467845917 val_loss: 0.08012460172176361
iter: 500 train_loss: 0.04585384950041771 val_loss: 0.07764910906553268
iter: 550 train_loss: 0.041616395115852356 val_loss: 0.0782717615365982
iter: 600 train_loss: 0.03824881836771965 val_loss: 0.07294777780771255
iter: 650 train_loss: 0.034795138984918594 val_loss: 0.07273538410663605
ite

### Evaluate test set

In [13]:
#Get logits on test_set

X_test2 = np.array(X_test)
y_test2 = torch.tensor(np.array(y_test))

images = []

for path in X_test2:
    image = Image.open(path)
    images.append(image)

#Pass through transformer

test_inputs = torch.cat(split(images, dim2))

#Output as desired

with torch.no_grad():

    #Set to eval mode
    classifier.eval()

    #Compute test losses

    test_logits, test_loss = classifier(test_inputs.to('cuda'), y_test2.to('cuda'))

    #Set back to train mode
    classifier.train()


In [14]:
cut_off = 0.5

normed_class_1 = F.softmax(test_logits, dim=1).to('cpu').numpy()[:, 1]
labels = []

for prob in normed_class_1:
    if prob > cut_off:
        labels.append(1)
    else:
        labels.append(0)

In [15]:
y_true = y_test2

In [16]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_true, labels).ravel()
tn, fp, fn, tp

(64, 2, 2, 61)

In [17]:
precision = tp / (tp + fp)
accuracy = (tp + tn) / (tp + fp + tn + fn)
recall = tp / y_true.numpy().sum()

In [18]:
accuracy, precision, recall

(0.9689922480620154, 0.9682539682539683, 0.9682539682539683)

### Compare to classifier that just picked by most frequent

In [19]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent')
clf.fit(X_test2, y_true)

DummyClassifier(strategy='most_frequent')

In [20]:
y_naive = clf.predict(X_test2)

In [21]:
tn, fp, fn, tp = confusion_matrix(y_true, y_naive).ravel()

In [22]:
precision = tp / (tp + fp)
accuracy = (tp + tn) / (tp + fp + tn + fn)
accuracy, precision

/tmp/ipykernel_92371/163943127.py:1: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


(0.5116279069767442, nan)